In [1]:
# Given the values of a single peakon on a coarse grid, this DNN predicts the value of the peakon at a particular location.
# Essentially, given the values shown by the @ and G symbols, and a location within the ... region, it will output its
# prediction for the value at that location.
#
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @
#
#       .........
#       .       .
# @   @ . @   @ . @   @
#       .       .
#       .       .
#       .       .
# @   @ . G   @ . @   @
#       .       .
#       .........
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @

In [2]:
import tensorflow as tf

from base import fenics as fc
from base import ml

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
### DNN hyperparameters
hidden_units = [100] * 7
logits = 1
processor = ml.ScaleCenterAtPoint()
activation=tf.nn.crelu

### Training hyperparameters #1
drop_rate = 0.0
drop_type = 'dropout'
gradient_clip = 1.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/point_general/'
uuid = None
model_dir = ml.model_dir_str(model_dir_base, hidden_units, logits,
                             processor=processor, activation=activation, 
                             uuid=uuid)

compile_kwargs = {'gradient_clip': gradient_clip,
                  'processor': processor,
                  'model_dir': model_dir}
dnn_factory = ml.DNNFactory(hidden_units, logits, drop_rate=drop_rate,
                            drop_type=drop_type, log_steps=log_steps, 
                            activation=activation, batch_norm=False,
                            compile_kwargs=compile_kwargs)

max_steps = 0
if TRAIN:
    # Start off without using FEniCS solutions because they're so much
    # slower to generate
    gen_one_data = fc.GenGeneralSolutionAtPoint(num_fenics=0)
    
    ### Training hyperparameters #2
    batch_size = 50
    max_steps += 5000
    learning_rate = 0.001
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory(learning_rate=learning_rate)
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)
    
    # Start using FEniCS solutions
    gen_one_data = fc.GenGeneralSolutionAtPoint()
    
    batch_size = 500
    max_steps += 3000
    learning_rate = 0.001
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory(learning_rate=learning_rate)
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)
        
    batch_size = 500
    max_steps += 2000
    learning_rate = 0.0001
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory(learning_rate=learning_rate)
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)
        
    batch_size = 500
    max_steps += 1000
    learning_rate = 0.00001
    with ml.BatchData.context(gen_one_data=gen_one_data, batch_size=batch_size) as train_input_fn:
        dnn = dnn_factory(learning_rate=learning_rate)
        dnn.train(input_fn=train_input_fn, max_steps=max_steps)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/point_multi_peakon/30x2_1__ScaleCenterAtPoint_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1a9bbf8828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./saved_models/point_mul